## License 

<span style="color:gray"> Copyright 2019 David Whiting and the H2O.ai team

<span style="color:gray"> Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

<span style="color:gray">     http://www.apache.org/licenses/LICENSE-2.0

<span style="color:gray"> Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

<span style="color:gray"> **DISCLAIMER:** This notebook is not legal compliance advice. </span>

<hr style="background-color: gray;height: 2.0px;"/>

# Introduction to H2O-3: Lesson 2

This is the second in a series of instructional Jupyter notebooks on H2O-3. While these notebooks are intended to be completely self-contained, an accompanying instructional video with additional commentary can be found <span style="color:red">**_here_** _(link to be added)_.</span>

These notebooks are built to be run on the <span style="color:blue">H2O.ai Aquarium&trade;</span> training platform under the `Coursework` lab. Please register for an account at [http://aquarium.h2o.ai](http://aquarium.h2o.ai). Registration and use of Aquarium is free.

<div style="margin-left: 3em;">

### Intended Audience

The target audience for this training notebook is data scientists, machine learning engineers, and other experienced modelers. Technically advanced analysts may also find this training useful. A working knowledge of python and some experience building statistical or machine learning models is assumed.


### Prerequisites

You should complete Lesson 1 before beginning this lesson.

### Learning Outcomes

By the end of this notebook, you will be able to ...

- [ ] Feature engineering: target encoding
- [ ] Load data directly into the H2O-3 cluster
- [ ] Use H2O-3 commands to perform basic data munging tasks 
- [ ] Engineer new data features
- [ ] Train and evaluate an XGBoost ML model
- [ ] Create and save a MOJO for model production
- [ ] Use H2O Flow for monitoring H2O-cluster activity and health
- [ ] Use H2O Flow for inspecting data and models

#### "Machine Learning" Outcomes

- [ ] Advanced XGBoost

</div>

<hr style="background-color: rgb(170,0,0);height: 2.0px;"/>


# Lending Club Loan Default

In this tutorial, we will go through a step-by-step workflow to determine loan deliquency. We will make predictions based only on the information available at the time the loan was issued.  The data for this exercise come from the public Lending Club data set, a description can be found [here](https://www.kaggle.com/pragyanbo/a-hitchhiker-s-guide-to-lending-club-loan-data/notebook).

## Workflow

1. Start the H2O-3 cluster
2. Import data
3. Clean data
4. Feature engineering
5. Model training
6. Examine model accuracy
7. Interpret model
8. Save and reuse model
9. AutoML (optional)
10. Stop H2O-3 cluster

# Step 1 (of ?). Start the H2O-3 cluster

The `os` commands below check whether this notebook is being run on the Aquarium platform. We use `h2o.init` command to connect to the H2O-3 cluster, starting it if it is not already up. The parameters used in `h2o.init` will depend on your specific environment.

In [1]:
import os
import h2o

startup = '/home/h2o/bin/aquarium_startup'
if os.path.exists(startup):
    os.system(startup)
    local_url = 'http://localhost:54321/h2o'
    aquarium = True
else:
    local_url = 'http://localhost:54321'
    aquarium = False

h2o.init(url=local_url)

Checking whether there is an H2O instance running at http://localhost:54321/h2o . connected.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,1 month and 30 days
H2O cluster name:,h2o
H2O cluster total nodes:,1
H2O cluster free memory:,13.98 Gb
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"accepting new members, healthy"


Note: The method you use for starting and stopping an H2O-3 cluster will depend on how H2O is installed and configured on your system. Remember that if you start a cluster, you will need to ensure that it is shut down when you are done.

# Step 2 (of ?). Import and update data

Just as we did in Lesson 1, import, clean, and transform the data using the following steps.

In [2]:
if aquarium:
    input_csv = "/home/h2o/data/lending_club/LoanStats3a.csv"
else:
    input_csv = "https://s3-us-west-2.amazonaws.com/h2o-tutorials/data/topics/lending/lending_club/LoanStats3a.csv"

loans = h2o.import_file(input_csv,
                        col_types = {"int_rate":"string", 
                                     "revol_util":"string", 
                                     "emp_length":"string", 
                                     "verification_status":"string"})

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
num_unfiltered_loans = loans.dim[0]

ongoing_status = ["Current",
                  "In Grace Period",
                  "Late (16-30 days)",
                  "Late (31-120 days)",
                  "Does not meet the credit policy.  Status:Current",
                  "Does not meet the credit policy.  Status:In Grace Period"
                 ]
loans = loans[~loans["loan_status"].isin(ongoing_status)]

num_filtered_loans = loans.dim[0]
num_loans_filtered_out = num_unfiltered_loans - num_filtered_loans

fully_paid = ["Fully Paid",
              "Does not meet the credit policy.  Status:Fully Paid"]
loans["bad_loan"] = ~(loans["loan_status"].isin(fully_paid))

loans["bad_loan"] = loans["bad_loan"].asfactor()

bad_loan_dist = loans["bad_loan"].table()
bad_loan_dist["Percentage"] = (100 * bad_loan_dist["Count"] / loans.nrow).round()

loans["int_rate"] = loans["int_rate"].gsub(pattern = "%", replacement = "") 
loans["int_rate"] = loans["int_rate"].trim()
loans["int_rate"] = loans["int_rate"].asnumeric()

loans["revol_util"] = loans["revol_util"].gsub(pattern="%", replacement="")
loans["revol_util"] = loans["revol_util"].trim()
loans["revol_util"] = loans["revol_util"].asnumeric()

loans["emp_length"] = loans["emp_length"].gsub(pattern="([ ]*+[a-zA-Z].*)|(n/a)", replacement="") 
loans["emp_length"] = loans["emp_length"].trim()

loans["emp_length"] = loans["emp_length"].gsub(pattern="< 1", replacement="0")
loans["emp_length"] = loans["emp_length"].gsub(pattern="10\\+", replacement="10")
loans["emp_length"] = loans["emp_length"].asnumeric()

loans["verification_status"] = loans["verification_status"].sub(pattern="VERIFIED - income source", 
                                                                replacement="verified")
loans["verification_status"] = loans["verification_status"].sub(pattern="VERIFIED - income", 
                                                                replacement="verified")
loans["verification_status"] = loans["verification_status"].asfactor()

loans["credit_length"] = loans["issue_d"].year() - loans["earliest_cr_line"].year()
loans["credit_length"].head()

loans["issue_d_year"] = loans["issue_d"].year()
loans["issue_d_month"] = loans["issue_d"].month().asfactor()

# Step 3 (of ?).  NLP Feature engineering

Feature engineering can be considered the "secret sauce" in building a superior predictive model: it is often (although not always) more important than the choice of machine learning algorithm. A very good summary of feature engineering recipes can be found in the online [Driverless AI Documentation](http://docs.h2o.ai/driverless-ai/latest-stable/docs/userguide/transformations.html). 

We will use NLP (natural language processing) to create word embedding features from the loan description text field in our data.

### Word Embeddings

One of the columns in our dataset is a user-provided description of why the loan was requested. The first few descriptions in the dataset are shown below.

In [5]:
loans["desc"].head()

desc
Borrower added on 12/22/11 > I need to upgrade my business technologies.
Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike. I only need this money because the deal im looking at is to good to pass up. Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike.I only need this money because the deal im looking at is to good to pass up. I have finished college with an associates degree in business and its takingmeplaces
""
"Borrower added on 12/21/11 > to pay for property tax (borrow from friend, need to pay back) & central A/C need to be replace. I'm very sorry to let my loan expired last time."
""
Borrower added on 12/16/11 > Downpayment for a car.
Borrower added on 12/21/11 > I own a small home-based judgment collection business. I have 5 years experience collecting debts. I am now going from a home office to a small office. I also plan to buy a small debt portfolio (eg. $10K for $1M of debt) My score is not A+ because I own my home and have no mortgage.
"Borrower added on 12/16/11 > I'm trying to build up my credit history. I live with my brother and have no car payment or credit cards. I am in community college and work full time. Im going to use the money to make some repairs around the house and get some maintenance done on my car. Borrower added on 12/20/11 > $1000 down only $4375 to go. Thanks to everyone that invested so far, looking forward to surprising my brother with the fixes around the house."
"Borrower added on 12/15/11 > I had recived a loan from Citi Financial about a year ago, I was paying 29.99 intrest, so the refinance is to cut that rate since cleaning up my credit I have been paying everything on time as shown on my credit report"
""


The descriptions may contain information that would assist in predicting default, but supervised learning algorithms in general have a hard time understanding text. We need to convert these strings into a numeric representation of the text in order for our algorithms to operate on it. There are multiple choices for doing so, in this example we will use the Word2Vec algorithm.

We start by defining stop words (terms that are considered too frequent to carry much information) 

In [6]:
STOP_WORDS = ["ax","i","you","edu","s","t","m","subject","can","lines","re","what",
              "there","all","we","one","the","a","an","of","or","in","for","by","on",
              "but","is","in","a","not","with","as","was","if","they","are","this","and","it","have",
              "from","at","my","be","by","not","that","to","from","com","org","like","likes","so"]

We next _tokenize_ the descriptions by breaking the text into individual words

In [7]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

words = tokenize(loans["desc"].ascharacter())

Then we train our Word2Vec model on the words extracted from our descriptions. We choose an output vector size of 100.

>What does Word2Vec do? At a high level, it is a dimensionality reduction method for numerical representations of text. But it reduces dimensionality while preserving relationships between words in the text.
>
>Suppose we were to create a dictionary of all the words in our descriptions, and further suppose that dictionary contained 2500 unique words. At one extreme, we could create an indicator variable for each word (i.e., one-hot encoding). This would yield 2500 new features that would certainly lead to massive overfitting of models.
>
>At the other extreme, suppose we had someone classify those words into different groups and create indicator variables for each group: e.g., `risky_words` ("bankruptcy", "default", "forfeit", "lien", etc.), `angry_words` (profanity, "complaint", etc.), and so on. This reduces dimensionality by manually grouping words, but it is extremely labor intensive.
>
>Word2Vec starts with the entire dictionary size $K$ as inputs and the selected vector size $k$ as the target number of outputs. Passing through the intermediate layer(s) of the Word2Vec neural net, a $k$-dimensional numeric representation of each word is derived. 

In [8]:
from h2o.estimators.word2vec import H2OWord2vecEstimator

w2v_model = H2OWord2vecEstimator(vec_size=100, model_id="w2v")
w2v_model.train(training_frame=words)

word2vec Model Build progress: |██████████████████████████████████████████| 100%


One way we quickly sanity check the Word2Vec model is by finding synonyms for specified words, e.g., "car"

In [9]:
w2v_model.find_synonyms("car", count=10)

OrderedDict([('auto', 0.7422929406166077),
             ('vehicle', 0.7028231620788574),
             ('prius', 0.6695665717124939),
             ('camry', 0.6531004905700684),
             ('chevy', 0.6526370048522949),
             ('truck', 0.6467186808586121),
             ('lexus', 0.6433395743370056),
             ('subaru', 0.6385617852210999),
             ('automobile', 0.6319136619567871),
             ('tahoe', 0.624847948551178)])

Assuming we are satisfied with our Word2Vec model results, we next calculate a vector for each description by averaging over all of the words in that description

In [10]:
desc_vecs = w2v_model.transform(words, aggregate_method="AVERAGE")
desc_vecs.head()

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
-0.0827432,-0.311592,0.0916469,-0.119461,-0.0670864,0.210824,0.237594,0.084594,-0.147895,0.0267837,-0.0483551,-0.0557261,0.120727,-0.156047,-0.309712,-0.023978,0.00722305,0.0306448,0.00429697,0.0623061,0.100103,0.0173946,0.0265167,0.0361656,0.0472754,-0.0829572,-0.282085,-0.0779016,0.0772898,0.0382716,-0.205816,0.261986,-0.19116,-0.0766714,-0.0877876,0.0594898,-0.0451095,-0.167086,0.203768,-0.00540842,0.182413,0.0499954,0.0425691,0.03574,-0.0565721,-0.119666,0.0189156,-0.0920114,-0.0246883,-0.0730491,0.0248127,0.121442,-0.0591111,0.11742,0.180158,0.133639,0.11118,0.0162391,-0.0613446,-0.123346,0.147424,-0.0748907,0.152998,-0.0109899,-0.0296048,-0.0256001,-0.0396424,0.0603905,0.2336,-0.0835621,0.184963,-0.0959705,-0.0465195,-0.0793141,-0.00786958,-0.131096,0.0754839,-0.0294643,-0.071675,-0.0364565,-0.0717914,0.182632,-0.111425,-0.0682433,0.0809352,0.0151769,-0.20785,0.204633,-0.109951,-0.0470898,-0.00664428,-0.033281,-0.136532,-0.0362196,0.265603,-0.144144,-0.0363123,-0.137763,-0.000297798,0.0971281
-0.042703,-0.266927,0.0667996,-0.223674,-0.0244438,0.161026,0.123476,0.05961,-0.106715,0.105301,-0.0809612,0.166925,0.0792761,-0.0910884,-0.177516,-0.0473783,0.0915445,0.0121603,-0.00314142,-0.10087,-0.0773362,0.0549768,0.0350505,0.0333728,-0.0739098,0.0606739,-0.308276,-0.0601732,0.0319071,0.0624162,-0.1689,0.0942023,-0.26958,-0.0916332,0.000407537,0.166706,0.0259836,-0.206251,0.0592816,0.107729,0.0636908,0.00220441,0.0202878,-0.113982,-0.0556164,-0.0451116,-0.00597455,-0.186629,-0.0211866,-0.158636,0.0657134,0.186828,0.0561265,0.0667466,0.300404,0.0283427,0.0473695,0.0469345,0.0241677,-0.0393244,0.0810989,0.00300378,0.121713,-0.0351339,-0.063187,-0.0258348,0.0466622,0.119279,0.0428098,-0.122688,0.152696,-0.0825322,0.0018682,-0.0386345,-0.0248463,-0.107219,-0.0793773,-0.00181596,-0.11146,-0.110977,-0.0150939,0.15418,0.0623848,0.0579595,0.159006,-0.137848,-0.103462,0.0354854,-0.10136,-0.0387062,-0.073366,-0.0359299,-0.135912,0.0441085,0.201908,-0.221087,0.0567038,-0.0714564,-0.0935563,0.0177299
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
-0.156906,-0.309844,0.0707774,-0.26208,-0.0101614,0.0814126,0.163429,0.109107,-0.0587882,0.106927,-0.0608258,0.144341,0.0911347,-0.0848769,-0.250078,-0.063564,0.0178131,-0.000557045,-0.0571395,-0.0767115,-0.00830751,0.0161359,-0.0282732,0.0398093,-0.0303693,-0.0599793,-0.29321,-0.0590218,-0.00801291,0.0409507,-0.0819052,0.115663,-0.111824,-0.0433147,-0.0349358,0.109148,-0.0220894,-0.233683,0.0849643,0.119454,-0.013634,-0.0292892,0.105649,-0.122573,-0.02541,-0.127681,-0.0891773,-0.0656652,-0.0488446,-0.0951372,-0.0109315,0.105248,0.00484259,0.0489804,0.218857,0.116777,0.13877,0.045073,0.0730338,-0.121298,0.0901677,-0.0496865,0.151495,-0.161853,-0.0565972,-0.0348895,0.113754,0.11751,0.0258257,-0.149406,0.185045,-0.0562983,-0.131476,-0.0312093,-0.0165813,-0.110901,-0.043998,-0.0611378,-0.0385125,-0.0581036,-0.0766229,0.204845,-0.0508468,0.0686639,0.213649,-0.109125,-0.115007,0.183448,-0.103882,-0.0673413,-0.0938222,-0.0293929,-0.0944009,0.00732803,0.0810202,-0.156518,0.141011,-0.173256,-0.0532413,0.102924
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na

Finally, we add the aggregated word embeddings from the Word2Vec model to the loans data

In [11]:
loans_w2v = loans
loans_w2v = loans_w2v.cbind(desc_vecs)

# Step 4 (of ?). Model training

Now that we have cleaned our data and added new columns, we train a model to predict bad loans. First split our loans data into train and test.

In [12]:
train, test = loans.split_frame(seed=25, ratios=[0.75])
train_w2v, test_w2v = loans_w2v.split_frame(seed=25, ratios=[0.75])

Next create a list of predictors as a subset of the columns of the `loans` H2O Frame

In [ ]:
cols_to_remove = ["initial_list_status",
                  "out_prncp",
                  "out_prncp_inv",
                  "total_pymnt",
                  "total_pymnt_inv",
                  "total_rec_prncp", 
                  "total_rec_int",
                  "total_rec_late_fee",
                  "recoveries",
                  "issue_d",
                  "collection_recovery_fee",
                  "last_pymnt_d", 
                  "last_pymnt_amnt",
                  "next_pymnt_d",
                  "last_credit_pull_d",
                  "collections_12_mths_ex_med" , 
                  "mths_since_last_major_derog",
                  "policy_code",
                  "loan_status",
                  "funded_amnt",
                  "funded_amnt_inv",
                  "mths_since_last_delinq",
                  "mths_since_last_record",
                  "id",
                  "member_id",
                  "desc",
                  "zip_code"]

predictors = list(set(loans.col_names) - set(cols_to_remove))

In [ ]:
predictors

Now create an XGBoost model for predicting loan default. This model is being run with almost all of the model-tuning values at their defaults. Later we may want to optimize the hyperparameters using a grid search. 

In [ ]:
from h2o.estimators import H2OXGBoostEstimator

param = {
      "ntrees" : 20
    , "nfolds" : 5
    , "seed": 25
}
xgboost_model = H2OXGBoostEstimator(**param)
xgboost_model.train(x = predictors,
                    y = "bad_loan",
                    training_frame=train_w2v,
                    validation_frame=test_w2v)

# Step 5 (of ?).  Examine model accuracy

The plot below shows the performance of the model as more trees are built.  This graph can help us see at what point our model begins overfitting.  Our test data error rate stops improving at around 8-10 trees.

In [ ]:
%matplotlib inline
xgboost_model.plot()

The ROC curve of the training and testing data are shown below.  The area under the ROC curve is much higher for the training data than the test data, indicating that the model is beginning to memorize the training data.

In [ ]:
print("Training Data")
xgboost_model.model_performance(train = True).plot()
print("Testing Data")
xgboost_model.model_performance(valid = True).plot()
print("X-Val")
xgboost_model.model_performance(xval=True).plot()

# Step 6 (of ?). Interpret model

The variable importance plot shows us which variables are most important to predicting `bad_loan`.  We can use partial dependency plots to learn more about how these variables affect the prediction.

In [ ]:
xgboost_model.varimp_plot(20)

As suspected, interest rate appears to be the most important feature in predicting loan default. The partial dependency plot of the `int_rate` predictor shows us that as the interest rate increases, the likelihood of the loan defaulting also increases.

In [ ]:
pdp = xgboost_model.partial_plot(cols=["int_rate"], data=train)

# Step 8 (of 10). Save and reuse model

The model can either be embedded into a self-contained Java MOJO package
or it can be saved and later loaded directly into an H2O-3 cluster. For production
use, we recommend using MOJO as it is optimized for speed. See the [guide](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/productionizing.html) for further information.

### Downloading MOJO

In [ ]:
xgboost_model.download_mojo()

### Save and reuse the model 

We can save the model to disk for later use.

In [ ]:
model_path = h2o.save_model(model=xgboost_model, force=True)
print(model_path)

After the H2O cluster shuts down, all unsaved data and models are lost. At some future date, we can load the model for batch scoring in the H2O cluster.

In [ ]:
loaded_model = h2o.load_model(path=model_path)

Using that model, we can also score new data with the predict function:

In [ ]:
bad_loan_hat = loaded_model.predict(test)
bad_loan_hat.head(15)

# Step 9 (of 10). AutoML (optional)

AutoML can be used for automating the machine learning workflow, which includes automatic training and tuning of many models within a user-specified time-limit or user specified model build limit. 

Stacked Ensembles will be automatically trained on collections of individual models to produce highly predictive ensemble models.

In [ ]:
from h2o.automl import H2OAutoML

aml = H2OAutoML(max_models=5, 
                max_runtime_secs_per_model=60, 
                include_algos = ["GLM", "DRF", "XGBoost", "StackedEnsemble"],
                seed=25)
aml.train(x=predictors, y='bad_loan', training_frame=train)

_While the AutoML job is running, you can open H2O Flow and monitor the model building process._

Once complete, the leaderboard contains the performance metrics of the models generated by AutoML:

In [ ]:
aml.leaderboard

Since we provided only the training H2O Frame during training, the models are sorted by their cross-validated performance metrics (AUC by default for classification). We can evaluate the best model (`leader`) on the test data:

In [ ]:
aml.leader.model_performance(test_data=test).plot()
aml.leader.model_performance(test_data=test)

Another convenient use of H2O Flow is to explore the various models built by AutoML.

# Step 10 (of 10). Stop H2O-3 cluster

In [ ]:
h2o.cluster().shutdown()

Once your work is completed, shutting down the H2O cluster frees up the resources reserved by H2O.

# Bonus: H2O-3 documentation

* http://docs.h2o.ai